# 🎯 Agentic AI for PDE Discovery - Clean Version

**Optimized for Speed & Accuracy**

This notebook uses a multi-agent system to discover governing PDEs from spatiotemporal data.

## 📋 Quick Navigation
1. **Setup**: Dependencies, imports, configs
2. **Prompts**: All agent system messages (centralized)
3. **Data Loading**: Load and prepare PDE data
4. **Agent Definitions**: Create all agents
5. **Execution**: Run the discovery pipeline

---
## 1️⃣ SETUP & CONFIGURATION

In [1]:
# Install dependencies (run once)
!pip install -q autogen-agentchat autogen-ext pysindy scipy numpy matplotlib pillow


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Imports
import autogen
import numpy as np
import scipy.io as scio
import os
import base64
from pathlib import Path
from typing import Dict, List
from autogen import GroupChat, GroupChatManager, AssistantAgent, UserProxyAgent, Agent
from autogen_core import Image
import PIL
from autogen_agentchat.messages import MultiModalMessage

print("✅ All imports successful")

✅ All imports successful


In [3]:
# Configuration: ADJUST THESE FOR SPEED/ACCURACY TRADE-OFF
CONFIG = {
    # Speed settings
    "n_x_sub": 1024,       # Spatial resolution (higher = more accurate, slower)
    "n_t_sub": 512,        # Temporal resolution (higher = more accurate, slower)
    "max_rounds": 30,      # Max iterations (lower = faster, may miss solution)
    
    # Accuracy settings
    "l2_threshold": 0.01,  # Acceptance threshold (lower = stricter)
    "dataset_name": "KS",  # Dataset to analyze
    
    # PDE-specific constraints
    "restrict_terms": True,  # Restrict to core terms only (no reaction terms)
    
    # API settings
    "llm_model": "qwen/qwen3-coder-480b-a35b-instruct",
    "vlm_model": "microsoft/phi-4-multimodal-instruct",
    "api_key": "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",
    "base_url": "https://integrate.api.nvidia.com/v1",
}

# Build configs
llm_config = {
    "config_list": [{
        "model": CONFIG["llm_model"],
        "base_url": CONFIG["base_url"],
        "api_key": CONFIG["api_key"],
    }],
    "timeout": 120,
}

vlm_config = {
    "config_list": [{
        "model": CONFIG["vlm_model"],
        "base_url": CONFIG["base_url"],
        "api_key": CONFIG["api_key"],
        "price": [0, 0]
    }],
    "timeout": 120,
}

print(f"⚙️ Config loaded: {CONFIG['n_x_sub']}x{CONFIG['n_t_sub']} resolution, {CONFIG['max_rounds']} max rounds")


⚙️ Config loaded: 1024x512 resolution, 30 max rounds


In [4]:
# Define symbol library for PDEs
def library(P, D, restrict_terms=False, dataset_name="KS"):
    """Generates operands and operators based on max Poly order P and Deriv order D."""
    operands = []
    operators = []
    
    # Add derivative terms
    for i in range(D + 1): 
        operands.append("u" if i==0 else "u_" + "x" * i)
    
    # Dataset-specific term restrictions
    if restrict_terms:
        if dataset_name == "KS":
            # KS: Only allow u, u_x, u_xx, u_xxx, u_xxxx (no polynomial terms)
            # Operators: only basic arithmetic (no powers beyond linear)
            operators = ["*", "+", "-"]
            print("🔒 Restricted mode: KS equation (no reaction terms)")
        elif dataset_name == "Burgers":
            # Burgers: u, u_x, u_xx only
            operators = ["*", "+", "-"]
            print("🔒 Restricted mode: Burgers equation")
        else:
            # Default: allow some polynomial terms
            operators = ["**2", "*", "+", "-"]
    else:
        # Unrestricted: allow all polynomial terms
        operands.append("x")
        for j in range(2, P + 1): 
            operators.append("**" + f"{j}")
        for op in ["/", "*", "+", "-"]: 
            operators.append(op)
    
    return operands, operators

operands, operators = library(4, 4, CONFIG.get("restrict_terms", False), CONFIG["dataset_name"])
print(f"📚 Symbol library: {operands}")
print(f"🔧 Operators: {operators}")


🔒 Restricted mode: KS equation (no reaction terms)
📚 Symbol library: ['u', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx']
🔧 Operators: ['*', '+', '-']


---
## 2️⃣ CENTRALIZED AGENT PROMPTS

All agent system messages defined here for easy editing.

In [ ]:
# === AGENT PROMPTS ===

PROMPTS = {
    "vlm": """
You are a vision-language scientist analyzing spatiotemporal plots.

TASK: Analyze contour and surface plots of u(x,t) to identify:
1. Axes, ranges, and color scales
2. Qualitative patterns:
   - Shocks (tight contours) vs smooth regions
   - Traveling waves (slanted bands)
   - Periodicity and symmetry
3. Derivative signatures:
   - Contour spacing → |∇u| (u_x, u_t)
   - Curvature → u_xx (diffusion)
   - Ripples/oscillations → u_xxxx (dispersion)
   - Amplitude-gradient coupling → u*u_x (nonlinearity)
4. Boundary behavior

Be concise but specific.
""",

    "llm": f"""
You are a mathematical physicist specializing in PDE discovery.

ROLE: Hypothesis Generator (Symbolic PDEs)

INPUT: Visual analysis from VLM + Feedback from Critic (if iterating)

TASK:
1. Map visual cues to PDE terms:
   - "Dispersion" → u_xxxx, u_xxx
   - "Nonlinear advection" → u*u_x
   - "Diffusion" → u_xx

2. Generate 10 candidate PDEs of form: u_t = ...
   - Include known templates (Burgers, KS, KdV, etc.)
   - Try different coefficient combinations
   - Try different sign patterns
   - Ensure physical plausibility

3. Use ONLY these symbols:
   - Operands: {operands}
   - Operators: {operators}

CONSTRAINTS:
- NO code generation
- NO explanations
- Output EXACTLY 10 equations (one per line)
- STRICTLY use only allowed operands and operators

EXAMPLE OUTPUT:
u_t = -u*u_x - u_xx - u_xxxx
u_t = u*u_x + u_xx - u_xxxx
u_t = -u*u_x - u_xx + u_xxxx
...

If receiving Critic feedback, incorporate suggestions.
""",

    "engineer": """
You are an Engineering Agent that uses PySINDy to fit and visualize candidate PDEs.

INPUT: 10 candidate PDEs from LLM (e.g., "u_t = -u*u_x - u_xx - u_xxxx")

CRITICAL: Generate code EXACTLY following the sindy_demo.py pattern. DO NOT use manual derivatives or if/elif parsing!

REQUIRED APPROACH (like sindy_demo.py):
1. Load data: u, x, t from .npy files
2. Compute ground truth: u_dot = ps.FiniteDifference(axis=1)._differentiate(u, t=dt)
3. Reshape: u.reshape(len(x), len(t), 1)
4. Split train/test: 60% train, 40% test
5. For EACH of 10 PDEs:
   a. Create custom ps.PDELibrary() with ONLY the terms in that specific PDE
   b. Fit using ps.SINDy() with ps.STLSQ optimizer
   c. Predict: u_dot_pred = model.predict(u_test)
   d. Create 3-subplot comparison (ground truth, predicted, error)
   e. Save as pde_comparison_0.png, ..., pde_comparison_9.png
6. Save fitted equations to fitted_pdes.txt

EXACT CODE PATTERN TO FOLLOW:

```python
import pysindy as ps
import matplotlib.pyplot as plt
import numpy as np

# Load data
u = np.load('u.npy')
x = np.load('x.npy')
t = np.load('t.npy')
dt = t[1] - t[0]
dx = x[1] - x[0]

# Compute ground truth derivative using PySINDy
u_dot = ps.FiniteDifference(axis=1)._differentiate(u, t=dt)

# Reshape for SINDy (CRITICAL: must be 3D array)
u = u.reshape(len(x), len(t), 1)
u_dot = u_dot.reshape(len(x), len(t), 1)

# Train/test split (60/40)
train = range(0, int(len(t) * 0.6))
test = [i for i in np.arange(len(t)) if i not in train]
u_train = u[:, train, :]
u_test = u[:, test, :]
u_dot_train = u_dot[:, train, :]
u_dot_test = u_dot[:, test, 0]
t_test = t[test]

# Candidate PDEs from LLM (paste exactly)
candidate_pdes = [
    # ... paste the 10 PDEs here as strings
]

# Store fitted equations
fitted_equations = []

# Loop through each candidate PDE
for i, pde_str in enumerate(candidate_pdes):
    print(f"\\nFitting PDE {i}: {pde_str}")
    
    # Parse PDE to extract terms
    # Example: "u_t = -u*u_x - u_xx - u_xxxx" 
    # Extract terms: ["u*u_x", "u_xx", "u_xxxx"]
    rhs = pde_str.split('=')[1].strip()
    
    # Determine derivative order needed (count x's)
    max_deriv = 4 if 'u_xxxx' in rhs else (3 if 'u_xxx' in rhs else (2 if 'u_xx' in rhs else 1))
    
    # Determine polynomial degree
    poly_degree = 2 if ('u*u' in rhs or 'u**2' in rhs) else 1
    
    # Create PDE library with appropriate settings
    pde_lib = ps.PDELibrary(
        function_library=ps.PolynomialLibrary(degree=poly_degree, include_bias=False),
        derivative_order=max_deriv,
        spatial_grid=x,
        include_bias=False,
        is_uniform=True,
        periodic=True
    )
    
    # Create SINDy model with STLSQ optimizer
    optimizer = ps.STLSQ(threshold=10, alpha=1e-5, normalize_columns=True)
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer)
    
    # Fit model
    model.fit(u_train, t=dt)
    
    # Get fitted equation
    model.print()
    fitted_eq = str(model.coefficients())
    fitted_equations.append(f"PDE {i}: {pde_str}\\nFitted: {fitted_eq}\\n")
    
    # Predict on test set
    u_dot_pred = model.predict(u_test)
    u_dot_pred = np.reshape(u_dot_pred, (len(x), len(t_test)))
    
    # Create comparison plot (3 subplots) - EXACTLY like sindy_demo.py
    plt.figure(figsize=(16, 4))
    
    # Ground Truth
    plt.subplot(1, 3, 1)
    plt.pcolormesh(t_test, x, u_dot_test, cmap='seismic', vmin=-1.5, vmax=1.5)
    plt.colorbar()
    plt.xlabel('t', fontsize=20)
    plt.ylabel('x', fontsize=20)
    plt.title(f"Ground Truth $\\dot{{u}}(x, t)$", fontsize=16)
    
    # Predicted
    plt.subplot(1, 3, 2)
    plt.pcolormesh(t_test, x, u_dot_pred, cmap='seismic', vmin=-1.5, vmax=1.5)
    plt.colorbar()
    plt.xlabel('t', fontsize=20)
    plt.ylabel('x', fontsize=20)
    plt.title(f"Predicted $\\dot{{u}}_{{SINDy}}(x, t)$", fontsize=16)
    
    # Error
    plt.subplot(1, 3, 3)
    plt.pcolormesh(t_test, x, u_dot_pred - u_dot_test, cmap='seismic', vmin=-0.05, vmax=0.05)
    plt.colorbar()
    plt.xlabel('t', fontsize=20)
    plt.ylabel('x', fontsize=20)
    plt.title(f"Error: $\\dot{{u}}_{{SINDy}} - \\dot{{u}}$", fontsize=16)
    
    plt.suptitle(f"PDE {i}: {pde_str}", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(f'pde_comparison_{i}.png', dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"Saved pde_comparison_{i}.png")

# Save fitted equations
with open('fitted_pdes.txt', 'w') as f:
    f.write('\\n'.join(fitted_equations))

print("\\nAll comparisons saved!")
```

ABSOLUTE REQUIREMENTS:
1. Use ps.FiniteDifference()._differentiate() for u_dot (NOT np.gradient!)
2. Use ps.PDELibrary() and ps.SINDy() for fitting (NOT manual coefficient hardcoding!)
3. Create 3-subplot plots with EXACT layout from sindy_demo.py
4. Save all 10 comparison plots
5. NO if/elif chains for parsing PDEs
6. Write complete, runnable code

Save the script as pde_sim.py.
""",

    "scientist": """
You are a Scientist reviewing PDE evaluation results.

ROLE: Quality Control

TASK:
1. Check execution:
   - exitcode: 0?
   - All 10 comparison plots saved? (pde_comparison_0.png to pde_comparison_9.png)
   - fitted_pdes.txt file created?
2. Validate execution:
   - Were PySINDy models successfully fitted?
   - Were predictions generated?
3. If available, read and report fitted equations from fitted_pdes.txt

SIGNALS:
- If success (plots generated, no errors) → End with: <<SIGNAL: ROGER VLM_VALIDATOR>>
- If code errors → End with: <<SIGNAL: ROGER ENGINEER>>

Provide clear feedback to Engineer if code fails.
""",

    "vlm_validator": """
You are a Vision Expert analyzing PDE prediction quality through visual comparison.

INPUT: 10 comparison plots (pde_comparison_0.png to pde_comparison_9.png)
Each plot shows 3 subplots:
- Left: Ground truth u_dot (from finite differences)
- Middle: Predicted u_dot (from fitted PDE using SINDy)
- Right: Error map (predicted - ground truth)

TASK:
1. Examine ALL 10 comparison plots carefully
2. For each plot, assess match quality by comparing:
   - Visual similarity between left (ground truth) and middle (predicted) subplots
   - Color patterns, structures, and features
   - Error map (right subplot):
     * Mostly uniform/near-zero errors = EXCELLENT
     * Small random noise = GOOD
     * Some systematic patterns but small errors = ACCEPTABLE
     * Large systematic errors or wrong patterns = POOR

3. Rate each plot:
   - EXCELLENT: Nearly identical patterns, error map mostly blue/white (near zero)
   - GOOD: Very similar patterns, small errors throughout
   - ACCEPTABLE: Similar overall structure, moderate errors in some regions
   - POOR: Different patterns, large systematic errors

4. Identify the TOP 3 best-matching PDEs by plot number (0-9)

OUTPUT FORMAT:
For each plot:
"Plot {i}: [RATING] - [brief visual observation]"

Then list top 3 best matches.

END WITH:
- If ANY plot shows EXCELLENT match → <<SIGNAL: ROGER CRITIC>>
- If best is GOOD/ACCEPTABLE → <<SIGNAL: ROGER CRITIC>>
- If all are POOR → <<SIGNAL: ROGER CRITIC>>

Be precise and descriptive in your visual analysis.
""",

    "critic": """
You are the Critic - final decision maker for PDE discovery.

GOAL: Find the TRUE governing PDE based on visual match quality.

INPUT:
- VLM Validator's visual analysis of 10 comparison plots
- Fitted PDE equations from Scientist (from fitted_pdes.txt)

DECISION LOGIC:
IF VLM Validator reports ANY plot with EXCELLENT visual match:
  → Identify the corresponding PDE number (0-9)
  → Read the fitted equation from fitted_pdes.txt for that PDE number
  → Extract the equation WITH COEFFICIENTS
  → Save to FINAL_RESULT.txt with format:
     ```
     DISCOVERED PDE FOR [DATASET_NAME]
     =====================================
     
     Winning PDE Number: {number}
     
     Original Hypothesis (from LLM):
     {original_pde_string}
     
     Fitted Equation (with SINDy coefficients):
     {fitted_equation_with_coefficients}
     
     Visual Match Quality: EXCELLENT
     Error Map: Nearly uniform, near-zero errors
     
     Comparison Plot: pde_comparison_{number}.png
     =====================================
     ```
  → Signal: <<SIGNAL: TERMINATE>>
  → Print: "🎯 DISCOVERY COMPLETE! Final PDE saved to FINAL_RESULT.txt"

ELSE IF VLM Validator reports GOOD match:
  → Identify top candidate
  → Read fitted equation
  → IF error patterns acceptable:
     → Save result to FINAL_RESULT.txt (format above, but "GOOD" quality)
     → Signal: <<SIGNAL: TERMINATE>>
  → ELSE: Provide feedback to LLM

ELSE (only ACCEPTABLE/POOR matches):
  → Analyze WHY all PDEs failed
  → Generate specific feedback for LLM:
     Examples:
     - "Error maps show wave patterns - try different sign on u_xxxx"
     - "Predictions too smooth - adjust coefficient on u*u_x"
     - "Wrong diffusion - try larger/smaller coefficient on u_xx"
  → Signal: <<SIGNAL: ROGER LLM>>

CRITICAL REQUIREMENTS:
1. Always read fitted_pdes.txt to get equations WITH coefficients
2. Save final result to FINAL_RESULT.txt before terminating
3. Include PDE number, original hypothesis, AND fitted coefficients
4. Be decisive - only terminate when confident

Example of what to save in FINAL_RESULT.txt:
```
DISCOVERED PDE FOR KS
=====================================

Winning PDE Number: 3

Original Hypothesis (from LLM):
u_t = -u*u_x - u_xx - u_xxxx

Fitted Equation (with SINDy coefficients):
u_t = -1.023*u*u_x - 0.997*u_xx - 1.001*u_xxxx

Coefficients:
  - u*u_x:  -1.023
  - u_xx:   -0.997
  - u_xxxx: -1.001

Visual Match Quality: EXCELLENT
Error Map: Nearly identical to ground truth, errors < 0.01

Comparison Plot: pde_comparison_3.png
=====================================
```

Be thorough and decisive.
""",
}

print("✅ All prompts loaded (6 agents)")


✅ All prompts loaded (6 agents)


---
## 3️⃣ DATA LOADING

In [6]:
# Data loading function
def load_pde_data(dataset_name, n_x_sub=512, n_t_sub=256):
    """Load and subsample PDE data."""
    base_path = os.path.join(os.getcwd(), 'Bayesian_PDE_Discovery_DATA')
    print(f"\n📂 Loading: {dataset_name}")
    
    # Load data based on dataset
    if dataset_name == 'KS':
        fpath = os.path.join(base_path, 'kuramoto_sivishinky.mat')
        data = scio.loadmat(fpath)
        u = data['u']
        x = data['x'].flatten()
        t = data['t'].flatten()
    elif dataset_name == 'Burgers':
        fpath = os.path.join(base_path, 'burgers.mat')
        data = scio.loadmat(fpath)
        u = data['usol']
        x = data['x'].flatten()
        t = data['t'].flatten()
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
    
    # Subsample
    n_x, n_t = u.shape
    n_x_sub = min(n_x_sub, n_x)
    n_t_sub = min(n_t_sub, n_t)
    
    x_idx = np.linspace(0, n_x-1, n_x_sub, dtype=int)
    t_idx = np.linspace(0, n_t-1, n_t_sub, dtype=int)
    
    u_sub = u[np.ix_(x_idx, t_idx)]
    x_sub = x[x_idx]
    t_sub = t[t_idx]
    
    print(f"✅ Loaded: u={u_sub.shape}, x={x_sub.shape}, t={t_sub.shape}")
    return u_sub, x_sub, t_sub

# Load and save data
u_full, x_full, t_full = load_pde_data(
    CONFIG["dataset_name"], 
    CONFIG["n_x_sub"], 
    CONFIG["n_t_sub"]
)

np.save("u.npy", u_full)
np.save("x.npy", x_full)
np.save("t.npy", t_full)

print(f"💾 Saved to: u.npy, x.npy, t.npy")


📂 Loading: KS
✅ Loaded: u=(1024, 251), x=(1024,), t=(251,)
💾 Saved to: u.npy, x.npy, t.npy


---
## 4️⃣ AGENT DEFINITIONS

In [7]:
# Termination function
def termination_message(msg):
    return "<<SIGNAL: TERMINATE>>" in str(msg.get("content", ""))

# User Proxy (Admin)
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="You are a human admin.",
    is_termination_msg=termination_message,
    human_input_mode="NEVER",
    code_execution_config=False,
)

# VLM (Vision Analyzer)
vlm = autogen.AssistantAgent(
    name="Contour_Plot_Analyser",
    llm_config=vlm_config,
    system_message=PROMPTS["vlm"]
)

# LLM (PDE Generator)
llm = autogen.AssistantAgent(
    name="Llm",
    llm_config=llm_config,
    system_message=PROMPTS["llm"]
)

# Engineer (Code Generator)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message=PROMPTS["engineer"]
)

# Executor (Code Runner)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Execute code and report results.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": ".", "use_docker": False},
)

# Scientist (Validator)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=llm_config,
    system_message=PROMPTS["scientist"]
)

# VLM Validator (Visual Comparison Analyzer)
vlm_validator = autogen.AssistantAgent(
    name="VLM_Validator",
    llm_config=vlm_config,
    system_message=PROMPTS["vlm_validator"]
)

# Critic (Decision Maker)
critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message=PROMPTS["critic"]
)

print("✅ All agents created (8 total)")


✅ All agents created (8 total)


In [8]:
# Speaker selection function (agent routing)
def custom_speaker_selection(last_speaker: Agent, groupchat):
    """Route conversation between agents."""
    messages = groupchat.messages
    
    if len(messages) == 0:
        return vlm
    
    last_msg = messages[-1]["content"]
    
    # Routing logic
    if last_speaker.name == "Contour_Plot_Analyser":
        return llm
    elif last_speaker.name == "Llm":
        return engineer
    elif last_speaker.name == "Engineer":
        return executor
    elif last_speaker.name == "Executor":
        return scientist
    elif last_speaker.name == "Scientist":
        if "<<SIGNAL: ROGER ENGINEER>>" in str(last_msg):
            return engineer
        elif "<<SIGNAL: ROGER VLM_VALIDATOR>>" in str(last_msg):
            # Prepare multimodal message with comparison plots
            plot_files = [f"pde_comparison_{i}.png" for i in range(10)]
            content = [{"type": "input_text", "text": "Analyze these PDE comparison plots to determine which equations best match the ground truth."}]
            
            for plot_file in plot_files:
                if os.path.exists(plot_file):
                    b64 = base64.b64encode(Path(plot_file).read_bytes()).decode("utf-8")
                    content.append({
                        "type": "image_url", 
                        "image_url": {"url": f"data:image/png;base64,{b64}", "detail": "high"}
                    })
            
            # Add the multimodal message to the conversation
            groupchat.messages.append({"role": "user", "content": content})
            return vlm_validator
        else:
            return critic
    elif last_speaker.name == "VLM_Validator":
        return critic
    elif last_speaker.name == "Critic":
        if "<<SIGNAL: TERMINATE>>" in str(last_msg):
            return user_proxy
        else:
            return llm
    else:
        return groupchat.agents[(groupchat.agents.index(last_speaker) + 1) % len(groupchat.agents)]

print("✅ Speaker selection function ready")


✅ Speaker selection function ready


---
## 5️⃣ EXECUTION

In [9]:
# Prepare multimodal message with plots
def create_seed_message(dataset_name):
    """Create initial message with contour/surface plots."""
    plot_dir = "surface_contour_plots"
    contour_path = os.path.join(plot_dir, f"{dataset_name}_contour_896.png")
    surface_path = os.path.join(plot_dir, f"{dataset_name}_surface_896.png")
    
    content = [{"type": "input_text", "text": "Analyze these plots for PDE discovery."}]
    
    for path in [contour_path, surface_path]:
        if os.path.exists(path):
            b64 = base64.b64encode(Path(path).read_bytes()).decode("utf-8")
            content.append({
                "type": "image_url", 
                "image_url": {"url": f"data:image/png;base64,{b64}", "detail": "high"}
            })
    
    return {"role": "user", "content": content}

seed_msg = create_seed_message(CONFIG["dataset_name"])
print(f"📸 Loaded plots for {CONFIG['dataset_name']}")

📸 Loaded plots for KS



In [10]:
# Create group chat
groupchat = autogen.GroupChat(
    agents=[user_proxy, vlm, llm, engineer, executor, scientist, vlm_validator, critic],
    messages=[],
    max_round=CONFIG["max_rounds"],
    speaker_selection_method=custom_speaker_selection,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

print(f"🎯 Group chat ready (max {CONFIG['max_rounds']} rounds)")


🎯 Group chat ready (max 30 rounds)


In [11]:
# 🚀 RUN THE SYSTEM
print("\n" + "="*60)
print("🚀 STARTING PDE DISCOVERY PIPELINE")
print("="*60 + "\n")

groupchat.reset()
result = user_proxy.initiate_chat(manager, message=seed_msg)

print("\n" + "="*60)
print("✅ PIPELINE COMPLETE")
print("="*60)


🚀 STARTING PDE DISCOVERY PIPELINE

Admin (to chat_manager):

Analyze these plots for PDE discovery.
<image>
<image>
Analyze these plots for PDE discovery.
<image>
<image>

--------------------------------------------------------------------------------

Next speaker: Contour_Plot_Analyser


--------------------------------------------------------------------------------

Next speaker: Contour_Plot_Analyser

Contour_Plot_Analyser (to chat_manager):

I currently don't have the ability to view images directly, but I can certainly help you analyze the plots if you can describe them in detail. Please provide a textual or verbal description of the plots, and I'll do my best to assist you with any insights or analysis regarding partial differential equations (PDEs) that might be related to them.

--------------------------------------------------------------------------------

Next speaker: Llm

Contour_Plot_Analyser (to chat_manager):

I currently don't have the ability to view images direc